In [427]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [428]:
base = pd.read_csv('iec6100429.csv')

In [429]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [430]:
x.head()

,Amostra 1,Amostra 2,Amostra 3,Amostra 4,Amostra 5,Amostra 6,Amostra 7,Amostra 8,Amostra 9,Amostra 10,...,Amostra 182,Amostra 183,Amostra 184,Amostra 185,Amostra 186,Amostra 187,Amostra 188,Amostra 189,Amostra 190,Amostra 191
0,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
1,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
2,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3
4,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,...,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3


In [431]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(56,)

In [432]:
y = y.values.reshape(-1,1)
y.shape

(56, 1)

In [433]:
y = onehot.fit_transform(y).toarray()

In [434]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.4)

In [435]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((56, 191), (23, 191), (56, 4), (23, 4))

In [436]:
neuronios_entrada = x.shape[1]
neuronios_entrada

191

In [437]:
neuronios_oculta = 150
neuronios_oculta

150

In [438]:
neuronios_saida = y.shape[1]
neuronios_saida

4

In [439]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable_76:0' shape=(191, 150) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_77:0' shape=(150, 4) dtype=float32_ref>}

In [440]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_78:0' shape=(150,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_79:0' shape=(4,) dtype=float32_ref>}

In [441]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [442]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [443]:
modelo = mlp(xph, w, b)

In [444]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(erro)

In [445]:
batch_size = 12
batch_total = int(len(x_treinamento) / batch_size)
batch_total

2

In [446]:
x_batches = np.array_split(x_treinamento, batch_total)

In [447]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(15000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])
    
    previsoes = tf.nn.softmax(mlp(xph, w_final, b_final))
    previsoes_corretas = tf.equal(tf.argmax(previsoes, 1), tf.argmax(yph, 1))
    taxa_acerto = tf.reduce_mean(tf.cast(previsoes_corretas, tf.float32))
    print(sess.run(taxa_acerto, feed_dict = {xph: x_teste, yph: y_teste}))

Época: 1 erro: 578.45263671875
Época: 501 erro: 2.916202876716852
Época: 1001 erro: 0.00015276086060111993
Época: 1501 erro: 9.470865256844263e-05
Época: 2001 erro: 6.307694177110079e-05
Época: 2501 erro: 4.348481203564347e-05
Época: 3001 erro: 2.890859222404174e-05
Época: 3501 erro: 1.858912432872728e-05
Época: 4001 erro: 1.167934040680052e-05
Época: 4501 erro: 7.235016937556793e-06
Época: 5001 erro: 4.4667012062404865e-06
Época: 5501 erro: 2.7711244712236294e-06
Época: 6001 erro: 1.7462641039855953e-06
Época: 6501 erro: 1.1495722915810802e-06
Época: 7001 erro: 8.250381853258659e-07
Época: 7501 erro: 6.017419096338017e-07
Época: 8001 erro: 4.533886226454342e-07
Época: 8501 erro: 3.387816889244277e-07
Época: 9001 erro: 2.6493351512613117e-07
Época: 9501 erro: 1.910852951425568e-07
Época: 10001 erro: 1.321381475349881e-07
Época: 10501 erro: 8.809212026505975e-08
Época: 11001 erro: 5.8728089946669115e-08
Época: 11501 erro: 3.308933571588568e-08
Época: 12001 erro: 2.563876044803237e-08
Ép

In [448]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))
    
y_teste2 = np.argmax(y_teste, 1)
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acerto

1.0